In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import json
import re

import seaborn as sns
import os

In [2]:
all_inpatients_file_path = r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\osn_vsetka_starostlivost_2023_ms_v2024.2.3.csv"

In [3]:
all_inpatients = pd.read_csv(all_inpatients_file_path, dtype=str, delimiter=";")

In [4]:
all_inpatients.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,osetrovacia_doba,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms,uroven_ms
0,35380_23000079,20,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-16,4,5130405,2002-04-25,522279,24,N61173014101,N6117301,N6117301,N61173,23000079,S05-34~S05-29,1~3
1,35380_23000214,19,NaN,NaN,NaN,j350,5g603&B&20230127,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-26,2023-01-31,5,2778465,2003-08-23,527696,24,N61173014101,N6117301,N6117301,N61173,23000214,S05-34~S05-29,1~3
2,35380_23000126,19,NaN,NaN,NaN,j350~j352,5g603&B&20230118~5g6071&Z&20230118,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-17,2023-01-21,4,3305786,2003-10-07,529249,24,N61173014101,N6117301,N6117301,N61173,23000126,S05-34~S05-29,1~3
3,35380_23000083,18,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-17,5,3305359,2004-08-15,520128,24,N61173014101,N6117301,N6117301,N61173,23000083,S53-33~S63-40,1~2
4,35380_23000159,18,NaN,NaN,NaN,h912,NaN,d61a,0.7048,DRG,h912,NaN,2023-01,2023-01,2023-01-20,2023-01-26,6,2852825,2004-08-31,520802,24,N61173014101,N6117301,N6117301,N61173,23000159,S53-29,3


In [5]:
pzs_gps = pd.read_excel(r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\other\PZS_zakl_info_GPS_bezprepojeni.xlsx", dtype=str)

In [6]:
pzs_gps.rename(columns={'IDENTIFZAR': 'pzs_8', 'SIDZAR_KOD_OBEC': 'KOD_OBEC_PZS', 'SIDZAR_KOD_OKRES_PZS': 'KOD_OKRES_PZS', 'lat_pzs_ifexists': 'lat_pzs', 'lng_pzs_ifexists': 'lng_pzs', 'NAZZAR': 'NAZOV_PZS', 'OSN_siet' : 'OSN_siet', 'OSN_level' : 'OSN_level', 'typ_nemocnice' : 'typ_nemocnice'}, inplace=True)

In [7]:
pzs_gps.head(2)

,pzs_8,PZS_short,NAZOV_PZS,OSN_siet,OSN_level,typ_nemocnice,KOD_OBEC_PZS,KOD_OKRES_PZS,lat_pzs,lng_pzs,NAZZAR_povodne,lat_pzs_ifexists_povodne,lng_pzs_ifexists.1,SIDZAR_KOD_OBEC_povodne,SIDZAR_KOD_OKRES_PZS_povodne
0,P5848001,P58480,"Ministerstvo vnútra Slovenskej republiky, všeo...",nie,mimo siet OSN,mimo siet OSN,501760,201,47.8001114823,17.6645174625,"Ministerstvo vnútra Slovenskej republiky, všeo...",47.8001114823,17.6645174625,501760,201
1,P5848002,P58480,"Ministerstvo vnútra Slovenskej republiky, všeo...",nie,mimo siet OSN,mimo siet OSN,520004,702,48.9336706585,21.9065813542,"Ministerstvo vnútra Slovenskej republiky, všeo...",48.9336706585,21.9065813542,520004,702


In [8]:
# Find the missing values
missing_values = set(all_inpatients['pzs_8']) - set(pzs_gps['pzs_8'])

# Convert the set to a list
missing_values_list = list(missing_values)

missing_values_list

[]

In [9]:
# Convert the list to a DataFrame
missing_values_df = pd.DataFrame(missing_values_list, columns=['missing_pzs_8'])

# Export to Excel
missing_values_df.to_excel('missing_values.xlsx', index=False)